In [1]:
import torch
from IPython.display import clear_output
import time

def knapsack_specialized(xi, v, w, C):
    """
    Implementation of the updated version of the algorithm.

    Parameters:
    xi: Ordered vector of size C.
    v: Ordered vector of size C.
    w: Real value belonging to v.
    C: Problem dimension.

    Returns:
    x_opt: Optimal solution (vector).
    lambda_opt: Lambda value.
    objective_values: Optimal value computed as the dot product between xi and x_opt.
    """
    
    b_list = []
    b = 0
    while True:
        delta_x = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_x / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break

    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    w_idx = torch.searchsorted(v, w)
    x_opt = torch.zeros(len(w), C, dtype=torch.float32)
    lambda_opt = torch.zeros(len(w), dtype=torch.float32)

    indices_1 = torch.nonzero(x_plus == 1).squeeze()
    indices_1_sorted = indices_1.sort().values

    left_positions = torch.searchsorted(indices_1_sorted, w_idx, right=False) - 1
    left_positions = torch.clamp(left_positions, min=0)

    right_positions = torch.searchsorted(indices_1_sorted, w_idx, right=True)
    right_positions = torch.clamp(right_positions, max=indices_1_sorted.size(0) - 1)

    idx_left = torch.where(x_plus[w_idx] == 1, w_idx, indices_1_sorted[left_positions])
    idx_right = torch.where(x_plus[w_idx] == 1, w_idx, indices_1_sorted[right_positions])

    # Handling the case where idx_left == idx_right
    mask_idx_equal = idx_left == idx_right
    not_c_minus_1_mask = (idx_left != (C - 1)) & mask_idx_equal
    is_c_minus_1_mask = (idx_left == (C - 1)) & mask_idx_equal

    if torch.any(not_c_minus_1_mask):
        idx_right[not_c_minus_1_mask] = indices_1_sorted[
            torch.searchsorted(indices_1_sorted, idx_left[not_c_minus_1_mask], right=True)
        ]

    if torch.any(is_c_minus_1_mask):
        idx_left[is_c_minus_1_mask] = indices_1_sorted[
            torch.searchsorted(indices_1_sorted, idx_right[is_c_minus_1_mask], right=False) - 1
        ]

    x1 = torch.zeros(len(w), C, dtype=torch.float32)
    x2 = torch.zeros(len(w), C, dtype=torch.float32)
    x1[torch.arange(len(w)), idx_left] = 1
    x2[torch.arange(len(w)), idx_right] = 1

    numerator = w - torch.matmul(x2, v)
    denominator = torch.matmul((x1 - x2), v)
    theta = numerator / denominator

    mask_equal = (x1 == x2)
    theta_expanded = theta.unsqueeze(1)
    x_opt = torch.where(mask_equal, x1, x1 * theta_expanded + x2 * (1 - theta_expanded))

    denominator = (v[idx_right] - v[idx_left])
    denominator_zero_mask = denominator == 0

    lambda_opt_nonzero = (xi[idx_right] - xi[idx_left]) / denominator
    lambda_opt_zero_full = xi / v
    lambda_opt_zero_full[0] = 0
    lambda_opt_zero = lambda_opt_zero_full[idx_left]

    lambda_opt = torch.where(denominator_zero_mask, lambda_opt_zero, lambda_opt_nonzero)

    objective_values = torch.matmul(x_opt, xi)

    return x_opt, lambda_opt, objective_values

# Initial parameters
max_iterations = 200
lambda_reg = 100  # Regularization parameter
C = 5
N = 3
upper_c = N
xi = torch.tensor([0.3451, 0.3661, 0.4231, 0.7080, 0.9156])
v = torch.linspace(0, (C - 1)/C, C)
w = torch.tensor([0.4, 0.6, 0])

# Variables for FISTA
xi_prev = xi.clone()
t_prev = 1.0

# Function to compute phi and the gradient
def compute_phi_and_gradient(xi, v, w, C, upper_c):
    """
    Compute the function phi and its gradient.

    Parameters:
    xi: Current solution vector.
    v: Ordered vector of size C.
    w: Real value belonging to v.
    C: Problem dimension.
    upper_c: Upper bound for constraint handling.

    Returns:
    phi: Computed phi value.
    g: Gradient vector.
    """
    x_i_star, lambda_plus, phi_plus = knapsack_specialized(xi, v, w, C)
    sum_x_star = torch.sum(x_i_star, dim=0)
    c_star = torch.exp(torch.log(torch.tensor(2.0)) * xi - 1)
    c_star = torch.clamp(c_star, min=0.01, max=upper_c)
    
    # Compute phi terms
    phi1 = torch.sum(c_star * torch.log(c_star) / torch.log(torch.tensor(2.0)))
    phi2 = -torch.sum(xi * c_star)
    phi3 = torch.sum(xi * sum_x_star)
    phi = phi1 + phi2 + phi3
    
    # Compute the gradient (super-gradient)
    g = -(c_star - sum_x_star)
    return phi, g

# Main FISTA loop
for iteration in range(1, max_iterations + 1):
    # Accelerated step (momentum)
    t_current = (1 + torch.sqrt(torch.tensor(1) + 4 * t_prev**2)) / 2
    y = xi + ((t_prev - 1) / t_current) * (xi - xi_prev)
    
    # Compute phi and gradient at y
    phi, g = compute_phi_and_gradient(y, v, w, C, upper_c)
    
    # Proximal update step
    xi_next = y + (1 / lambda_reg) * g  # Proximal update
    
    # Clipping to ensure constraints on xi
    xi_next = torch.clamp(xi_next, min=0.01, max=upper_c)
    
    # Display progress
    clear_output(wait=True)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    time.sleep(0.01)
    
    # Update for the next iteration
    xi_prev = xi.clone()
    xi = xi_next.clone()
    t_prev = t_current


Iteration 200: phi = -2.0198259353637695


In [2]:
import torch
from torch.linalg import norm
from IPython.display import clear_output
import time

# Initial parameters
max_iterations = 1000
lambda_reg = 100  # Regularization parameter
C = 5
N = 3
upper_c = N
xi = torch.tensor([0.3451, 0.3661, 0.4231, 0.7080, 0.9156], requires_grad=False)
v = torch.linspace(0, (C - 1)/C, C)
w = torch.tensor([0.4, 0.6, 0], requires_grad=False)

# Parameters for the Bundle method
epsilon = 1e-5  # Convergence tolerance
bundle_size = 5  # Maximum bundle size
bundle = []  # Initial bundle (list of points, phi values, and gradients)

# Main loop of the Bundle Proximal method
for iteration in range(1, max_iterations + 1):
    # Compute phi and gradient at the current point
    phi, g = compute_phi_and_gradient(xi, v, w, C, upper_c)
    
    # Add the current point to the bundle
    bundle.append((xi.clone(), phi, g.clone()))
    if len(bundle) > bundle_size:
        bundle.pop(0)  # Remove the oldest point if the bundle exceeds the maximum size
    
    # Solve the subproblem (regularized quadratic minimization)
    bundle_points = torch.stack([item[0] for item in bundle])  # Bundle points
    bundle_phis = torch.tensor([item[1] for item in bundle])  # Phi values
    bundle_gradients = torch.stack([item[2] for item in bundle])  # Bundle gradients
    
    # Construct the regularized quadratic problem
    diff = xi - bundle_points
    model_phi = bundle_phis + torch.sum(bundle_gradients * diff, dim=1)
    proximal_term = (lambda_reg / 2) * norm(diff, dim=1)**2
    subproblem_objective = model_phi + proximal_term
    
    # Find the new point by minimizing the model
    best_idx = torch.argmin(subproblem_objective)
    xi_next = bundle_points[best_idx] + (1 / lambda_reg) * bundle_gradients[best_idx]
    
    # Clipping to ensure constraints on xi
    xi_next = torch.clamp(xi_next, min=0.01, max=upper_c)
    
    # Convergence check
    clear_output(wait=True)
    time.sleep(0.01)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    
    if norm(xi_next - xi) < epsilon:
        print("Convergence reached!")
        break
    
    # Update xi
    xi = xi_next.clone()

Iteration 1000: phi = -2.0054593086242676
